**Outward Normal Method of approximating minimal invariant set**

Starting from a sample on the boundary of an epsilon ball of a point in the attractor, we compute the subsequent boundary
under the set-valued mapping $F(x) = \overline{B_{\varepsilon}(f(x))}$, i.e. a deterministic mapping with an additive term 
contained in the ball of size epsilon. 

By setting the formula of the original map in (func_sym) and its parameters, the program uses the boundary mapping which 
keep track of the boundary point and its outward pointing normal vector. Then, the fixed (or periodic) points on the boundary 
according to the boundary map are determined by applying the inverse boundary map. The inverse of the original function is
needed in (func_sym_inv).

The subsequent boundary is calculated by using points on the initial epsilon ball $\partial B_{\varepsilon}(x_0)$ of an initial
point $x_0$. Hence, depending on the boundary mapping, two very close point on the initial boundary might be needed to fill
the gaps on the boundary of nth iteration of the set boundary where n is large. Thus, we use mpmath to implement arbritary 
precision of float number.

Challenges: 
- tolerance between point need to be small for accuracy but not too small for memory. 
- Delete points less than epsilon-epstol away from image, epstol need to be small for successful removal of interior points
  but not too small to preserve enough points near the singularities. In some scenario, unstable points could be near the
  singularity, thus deleting too much points could leads to sample points only on one side of the unstable boundary point
  leading to gap appears on the boundary over time.

In [ ]:
'''Definitions of functions and boundary mapping'''
import sympy as sp
import numpy as np
from numpy import sin, cos
import time
import scipy.optimize as opt
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import sys

start_time = time.time()
# Define functions
a, b, x, y, n1, n2, eps = sp.symbols('a b x y n1 n2 eps')
c,d,k = sp.symbols('c d k')
func_sym = [1+y-a*x**2,b*x] # Deterministic map (in this case Henon map)
func_sym_jacinv = (sp.Matrix(func_sym).jacobian([x,y])**-1).T # InvTransp jacobian symbolic
func_norm = func_sym_jacinv*sp.Matrix([n1,n2]) # InvTransp jacobian multiply by n

# inverse mapping
func_sym_inv = [y/b,x+a/b**2*y**2-1] # Inverse henon symbolic
func_inv = sp.lambdify([(x,y,a,b,eps)],func_sym_inv) # inverse lambda function

# boundary mapping
func_norm_unit = [func_norm[0]/(func_norm[0]**2+func_norm[1]**2)**(1/2),
                  func_norm[1]/(func_norm[0]**2+func_norm[1]**2)**(1/2)]
x1 = [func_sym[0]+eps*func_norm_unit[0],func_sym[1]+eps*func_norm_unit[1]] # One step of extended map
F_sym = sp.Array([x1[0],x1[1],func_norm_unit[0],func_norm_unit[1]])

# Create lambda functions for original map and the boundary map
F_func = sp.lambdify([(x,y,n1,n2,a,b,eps)],F_sym,modules={'atan': np.arctan})
FFdet_sym = sp.Array([func_sym[0],func_sym[1]]) # deterministic without normal
FFdet_func = sp.lambdify([(x,y,a,b,eps)],FFdet_sym,modules={'atan': np.arctan})
Fdet_sym = sp.Array([func_sym[0],func_sym[1],func_norm_unit[0],func_norm_unit[1]])#determnistic
Fdet_func = sp.lambdify([(x,y,n1,n2,a,b,eps)],Fdet_sym,modules={'atan': np.arctan})

# inverse of boundary mapping
xx = [x-eps*n1,y-eps*n2]
x1 = func_inv([xx[0],xx[1],a,b,eps])
func_sym_inv_norm = (sp.Matrix(func_sym_inv).jacobian([x,y])**-1).T # InvTrans jacobian of inverse symbolic
func_sym_inv_norm = func_sym_inv_norm.subs([(x,xx[0]),(y,xx[1])])
func_inv_norm = [func_sym_inv_norm[0,0]*n1+func_sym_inv_norm[0,1]*n2,
    func_sym_inv_norm[1,0]*n1+func_sym_inv_norm[1,1]*n2] # InvTrans jacobian of inverse multiply by n func
F_inv_sym = sp.Array([x1[0],x1[1],func_inv_norm[0]/(func_inv_norm[0]**2+func_inv_norm[1]**2)**(1/2),func_inv_norm[1]/(func_inv_norm[0]**2+func_inv_norm[1]**2)**(1/2)])
F_inv_func = sp.lambdify([(x,y,n1,n2,a,b,eps)],F_inv_sym)

# Defines functions
def ff(xx): #deterministic without normal
    x,y = xx[0], xx[1]
    return np.array(FFdet_func([x,y,a_sub,b_sub,epsilon]))
def F(xx): # extended map with parameters a_sub and b_sub
    x, y, n1, n2 = xx[0], xx[1], xx[2], xx[3]
    return np.array(F_func([x,y,n1,n2,a_sub,b_sub,epsilon]))
def F_n(xx,n): # n-steps of extended
    for i in range(n):
        xx = F(xx)
    return xx

# Inverse functions 
def f_inv(xx): # deterministic mapping
    x, y = xx[0], xx[1]
    return np.array(func_inv([x,y,a_sub,b_sub,epsilon]))
def F_inv(xx): # inverse boundary map with a_sub and b_sub
    x, y, n1, n2 = xx[0], xx[1], xx[2], xx[3]
    return np.array(F_inv_func([x,y,n1,n2,a_sub,b_sub,epsilon]))
def F_invn(xx,n): # n periodic equation 
    xx1 = xx
    for i in range(n):
        xx = F_inv(xx)
    return xx

print('elapsed time is {}s.'.format(time.time()-start_time))

In [ ]:
'''
Outward normal method using boundary mapping

'''

from matplotlib import animation
from mpmath import mp
%matplotlib inline
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm
num_cores = multiprocessing.cpu_count() # numbers of cpu cores for parallel computations

start_time = time.time()

# initialise arrays
all_data, all_image = {}, {}
data_inv_recurrent, data_recurrent = {}, {}
boun_len_all = {}
bounmap_all, bounmap_inv_all, boun_iter_all = {}, {}, {}
sin,cos = np.vectorize(mp.sin), np.vectorize(mp.cos) # vectorise mpmath trigo operations

# set up initial values and parameters
epsilon = 0.0625 # size of the ball
epsaddx, epsaddy = 0.0002, 0.0002 # adjust on the initial epsilon ball for faster convergence
mp.dps = 50 # precision of array
epstol = 1e-6 # adjustment of epsilon for combat numerical error
a_ini = 0.6 # starting parameter a
a_final = 0.6 # ending parameter a
a_sub, b_sub = a_ini, 0.3
iter1 = 1 # total iteration of parameter
n_sample = 1000 # initial points on the ball 
total_iter = 60 # max iterations of algorithm
tolerance = 4e-3 # tol for dist between points
bound_tol = 1e-8 # convergence tolerence for the boundary length
doubledelete = 0 #check for points to delete second time
max_add = 100 # limit of adding of points
inner_miss = .05 # the limit to where interior points wrongly added 
plot_on = 0 # whether to plot or not
print_on = 0 # whether to print details
plot_one = 1 # plot one plot at the end
kron_on = 0 # whether to use np.kron, avoid when data large

x_ini = np.array([0.4,-0.2]) # initial value
data_boun = np.array([[x_ini[0]],[x_ini[1]]])
conv_len = 14 # the length of iter to monitor

for j in range(iter1):
    x_ini = np.mean(data_boun[:2,:], axis=1)
    all_data_iter, all_image_iter, all_derivative_iter = {}, {}, {} # record trajectories
    boun_len_iter = [] # recording all boundary length
    
    # start from recurrent set (get rid of transient)
    for i in range(100): # run for 100 iterations until converge
        x_ini1 = np.array(ff(x_ini))
        chg, x_ini= np.linalg.norm(x_ini1-x_ini),x_ini1
        if any(x_ini>10): #prevent diverge
            print('initial points diverge at x = {}, y = {}'.format(x_ini[0],x_ini[1]))
            sys.exit()
            
    # initial epsilon ball
    theta = np.array(mp.linspace(0,2*np.pi,n_sample))
    theta1 = np.array(theta.tolist(), dtype=float)
    data_boun = np.array([(epsilon+epsaddx)*np.cos(theta1)+x_ini[0],(epsilon+epsaddy)*np.sin(theta1)+x_ini[1],np.cos(theta1),np.sin(theta1)])
    immune = np.array([]) # for points near singularity immuned to adding points

    # Start iterations
    for i in range(total_iter):
            
        # map the boundary points
        image = ff(data_boun) # mapping without noise f
        data_boun = F(data_boun) # one step of boundary map
        data_boun1 = np.array(data_boun[:2,:]) # without normal
            
        if (np.abs(data_boun)>10).any():
            print('divergence of points')
            sys.exit()
            
        if i == total_iter - 7: 
            epstol = 1e-9 # reduce the eps tolerance in last two iteration
            tolerance /= 3  # reduce the gap for the last two iteration
            
        # Add points on boundary (minus ind with ind-1)
        dist_data = np.linalg.norm(data_boun1[:,1:]-data_boun1[:,:-1],axis=0) # distance between points
        find_ind = np.where(np.logical_and(dist_data>tolerance,dist_data<inner_miss))[0]
        find_ind = np.setdiff1d(find_ind,immune) # remove immuned singularity array if there are any
        add_iter = 1 # initialise iteration number in adding points
        
        if plot_on:
            plt.plot(data_boun[0,:],data_boun[1,:],'.',markersize=1)
            plt.plot(image[0,:],image[1,:],'.',markersize=1)
            plt.plot(data_boun[0,np.int32(immune)],data_boun[1,np.int32(immune)],'.',markersize=10)
            plt.plot(data_boun[0,find_ind],data_boun[1,find_ind],'.',markersize=5)
            plt.title('Step {} at a = {} after initial cut'.format(i,a_sub))
            plt.axis('equal')
            plt.show()
        
        while len(find_ind)>0 and add_iter < max_add:
            param_add = theta[find_ind]+(theta[find_ind+1]-theta[find_ind])/2 # add parameters
            if i < 23: # use numpy if still can
                param_add1 = np.array(param_add.tolist(),dtype=float)
                start_p = np.array([(epsilon+epsaddx)*np.cos(param_add1)+x_ini[0],(epsilon+epsaddy)*np.sin(param_add1)+x_ini[1],np.cos(param_add1),np.sin(param_add1)])
            else:
                start_p = np.array([(epsilon+epsaddx)*cos(param_add)+x_ini[0],(epsilon+epsaddy)*sin(param_add)+x_ini[1],cos(param_add),sin(param_add)])
            add = F_n(start_p,i+1) # i+1th iteration from epsilon circle
            if i >= 23: # change the elements back to float
                add = np.array(add.tolist(),dtype=float)
            theta = np.insert(theta,find_ind+1,param_add) # update parameters
            data_boun = np.insert(data_boun,find_ind+1,add,axis=1) # update parameters
            data_boun1 = np.array(data_boun[:2,:])
            for ii in range(len(immune)): # update immune array after adding points
                immune[ii] += len(np.where(find_ind<immune[ii])[0])
            
            # detect wrongly assigned point in the interior(through singularity)
            dist_data = np.linalg.norm(data_boun1[:,1:]-data_boun1[:,:-1],axis=0)
            miss_ind = np.where(dist_data>inner_miss)[0] # conclude interior points if more than some threshold
            miss_ind = np.setdiff1d(miss_ind,immune) # remove immuned singularity array if there are any
            while len(miss_ind) > 1:
                miss_array = []
                for miss in range(len(miss_ind)):
                    if miss < len(miss_ind)-1:
                        if miss_ind[miss+1]-miss_ind[miss] == 1: # back and front exceed limit
                            miss_array.append(miss_ind[miss+1])
                    for kkk in range(len(immune)): # add the point as a singularity
                        if immune[kkk]>miss_ind[miss]:
                            immune = np.insert(immune,kkk,miss_ind[miss])
                        elif immune[kkk]==miss_ind[miss]:
                            break
                        elif kkk == len(immune)-1:
                            immune = np.insert(immune,kkk+1,miss_ind[miss])
                miss_array = np.array(miss_array)
                if len(miss_array) > 0:
                    data_boun = np.delete(data_boun,miss_array,axis=1)
                    data_boun1 = np.array(data_boun[:2,:])
                    theta = np.delete(theta,miss_array)
                    for ii in range(len(immune)): # update immune array
                        immune[ii] -= len(np.where(miss_array<immune[ii])[0])
                dist_data = np.linalg.norm(data_boun1[:,1:]-data_boun1[:,:-1],axis=0)
                miss_ind = np.where(dist_data>inner_miss)[0]
                miss_ind = np.setdiff1d(miss_ind,immune) # remove immuned singularity array if there are any
                print(miss_ind)
            
            # calculate the distance for adding points
            dist_data = np.linalg.norm(data_boun1[:,1:]-data_boun1[:,:-1],axis=0)
            find_ind = np.where(np.logical_and(dist_data>tolerance,dist_data<inner_miss))[0] # large distance but not too far
            find_ind = np.setdiff1d(find_ind,immune) # remove immuned singularity array if there are any
            add_iter += 1
            
            if plot_on:
                plt.plot(data_boun[0,:],data_boun[1,:],'.',markersize=1)
                plt.plot(data_boun[0,find_ind],data_boun[1,find_ind],'.',markersize=5)
                plt.plot(data_boun[0,np.int32(immune)],data_boun[1,np.int32(immune)],'.',markersize=10)
                plt.title('Step {} add point iter {} with points added {}'.format(i,add_iter,len(param_add)))
                plt.axis('equal')
                plt.show()
    
        if print_on:
            print('elapsed time after adding points is {} with size {}'.format(time.time()-start_time,np.shape(data_boun)[1]))
        
        # record distance data before cut
        dist_data1 = np.linalg.norm(data_boun1-np.roll(data_boun1,1,axis=1),axis=0)
        
        # Delete points less than epsilon away from image
        data_len, image_len = np.shape(data_boun1)[1], np.shape(image)[1]
        if kron_on == 1: # use kron when size of data not too large
            check_cut = np.linalg.norm(np.kron(data_boun1,np.ones((1,image_len)))
                                       - np.kron(np.ones((1,data_len)),image),axis=0)
            cut_ind = np.unique(np.where(check_cut<epsilon-epstol)[0]//image_len) # minus a bit from epsilon to compensate with non-perfect boundary
        else: # use for loop when data too large for kron
            cut_ind = np.array([k for k in range(data_len)])
            def cut_func(kron_iter):
                if not(np.any(np.linalg.norm(data_boun1[:,kron_iter]-np.transpose(image),axis=1)<epsilon-epstol)):
                    return False
                else:
                    return True
            inputs = tqdm(cut_ind)
            if __name__ == "__main__":
                processed_list = Parallel(n_jobs=num_cores)(delayed(cut_func)(i) for i in inputs)
            cut_ind = cut_ind[processed_list]
            
        immune = np.array([]) # for points near singularity immuned to adding points
        if len(cut_ind)>0:
            data_boun = np.delete(data_boun,cut_ind,axis=1)
            data_boun1 = np.array(data_boun[:2,:])
            theta = np.delete(theta,cut_ind)
            immune = []
            for dlt in range(len(cut_ind)-1): #check for singularity points
                if dlt == 0 and cut_ind[0]!=0:
                    immune.append(cut_ind[dlt]-1)
                elif cut_ind[dlt+1]-cut_ind[dlt] > 10: # end point of singularity
                    immune.append(cut_ind[dlt+1]-dlt-2)
            immune = np.array(immune)
            #print('immune cell is {} at {}'.format(data_boun[:2,immune],immune)) # for debug
            

        # Delete missed points on the interior of set more than epsilon away from image 
        dist_data = np.linalg.norm(data_boun1-np.roll(data_boun1,1,axis=1),axis=0)
        interior = np.where(dist_data > max(dist_data1))[0] # detect points got further away from original boundary
        if len(interior) == 2: # non-empty interior points
            if np.abs(interior[0]-interior[1])<np.shape(data_boun1)[1]/2: # determine which section is interior
                delete_interior = np.arange(interior[0],interior[1])
            else:
                delete_interior = np.arange(interior[1],np.shape(data_boun1)[1],1)
                delete_interior = np.insert(delete_interior,0,np.arange(0,interior[0]),axis=0)
            data_boun = np.delete(data_boun,delete_interior,axis=1)
            data_boun1 = np.array(data_boun[:2,:])
            theta = np.delete(theta,delete_interior)
            wrong_ind = []
            for dlt in range(len(immune)): #check if wrongly classify interior point as singularities
                if any(delete_interior==immune[dlt]):
                    wrong_ind.append(dlt)
                else:
                    immune[dlt] -= len(np.where(delete_interior<immune[dlt])[0])
            immune = np.delete(immune,wrong_ind)        
        else:
            delete_interior = []    
        if print_on:
            print('deleted points on the interior, size {}, cut {} and {}'.format(np.shape(data_boun)[1],len(cut_ind),len(delete_interior)))
            print('elapsed time is {}'.format(time.time()-start_time))
        
        if plot_on or plot_one:
            plt.plot(data_boun[0,:],data_boun[1,:],'.',markersize=1)
            plt.plot(data_boun[0,:],data_boun[1,:])            
            plt.plot(image[0,:],image[1,:],'.',markersize=1)
            plt.plot(data_boun[0,np.int32(immune)],data_boun[1,np.int32(immune)],'.',markersize=10)
            plt.title('Step {} at a = {}'.format(i,a_sub))
            plt.axis('equal')
            plt.show()
        
        # Record data for plotting
        all_data_iter[str(i)] = data_boun
        all_image_iter[str(i)] = image
        
        # Monitor convergence
        if i == 0:
            len_record = np.zeros(conv_len)
        boun_diff = np.linalg.norm(data_boun1-np.roll(data_boun1,1,axis=1),axis=0)
        boun_length = np.sum(boun_diff[boun_diff<inner_miss]) # only count close distance between points
        boun_len_iter.append(boun_length)
        
        print('length of boundary is {} at step {} with size {} at time {}'.format(boun_length,i,np.shape(data_boun)[1],time.time()-start_time))
        # break the loop when length of MIS converges
        stop_ind = np.where(np.abs(len_record-boun_length)<bound_tol)[0] # boundary length match in one of the 5 iterations
        len_record = np.roll(len_record,1)
        len_record[0] = boun_length
        if (i>1 and len(stop_ind)>1) or i == total_iter-1: 
            if i == total_iter-1:
                stop_ind = 7 # manually assign periodicity if boundary lengths do not converge
                print('conclude that periodicity is {}'.format(stop_ind))
            else:
                stop_ind = stop_ind[0]
            data_final = data_boun
            image_final = image
            for jj in range(1,stop_ind+1): # find period of the MIS components
                if np.min(np.linalg.norm(all_data_iter[str(i-jj)][:2,:]-np.reshape(data_boun1[:,0],(2,1)),axis=0)) < tolerance*10:
                    stop_ind = jj-1
                    break
            if stop_ind > 0: # if for example the boundary consist of multiple cconnected omponents
                for ii in range(1,stop_ind+1):
                    data_final = np.insert(data_final,np.int32(np.zeros(np.shape(all_data_iter[str(i-ii)])[1])),
                                          all_data_iter[str(i-ii)],axis=1)
                    image_final = np.insert(image_final,np.int32(np.zeros(np.shape(all_image_iter[str(i-ii)])[1])),
                                          all_image_iter[str(i-ii)],axis=1)
            print('minimal invariant set found at step {}'.format(i))
            print('size of the data is {}'.format(np.shape(data_final)[1]))
            break
        elif i==total_iter-1:
            data_final = data_boun
            image_final = image
            break
        
    # record final boundary in each parameter
    all_data[str(j)] = data_final
    all_image[str(j)] = image_final
    boun_len_all[str(j)] = boun_len_iter
    
    
    ## find unstable points on the boundary        
    data_len = np.shape(data_final)[1]
    pt_conv = 7 # minimum number of data converge to unstable point before concluding
    traj_iter = data_len
    data_final1 = F_inv(data_final) # one step of inverse boundary map
    if kron_on: # use kron when final data not too large
        big_norm = np.linalg.norm(np.kron(data_final1[:2,:],np.ones((1,data_len))) - np.kron(np.ones((1,data_len)),data_final[:2,:]),axis=0)
        boun_map_inv = np.argmin(np.reshape(big_norm,(data_len,data_len)),axis=1) # find the closest data point on boundary hit by inverse boun map
    else: # use for loop when data too large for kron
        def boun_func_inv(kron_iter):
            return np.argmin(np.linalg.norm(data_final1[:2,kron_iter]-np.transpose(data_final[:2,:]),axis=1))
        inputs = tqdm(range(data_len))
        if __name__ == "__main__":
            boun_map_inv = np.array(Parallel(n_jobs=num_cores)(delayed(boun_func_inv)(i) for i in inputs))
        
    boun_iter = np.array(boun_map_inv)
    boun_len = len(np.unique(boun_iter))
    for p in range(traj_iter): # repeat the boundary map
        boun_iter = boun_map_inv[boun_iter]
        if len(np.unique(boun_iter)) == boun_len:
            break
        boun_len = len(np.unique(boun_iter))

    # Record boundary dynamic data for plotting
    bounmap_inv_all[str(j)] = boun_map_inv
    boun_unique = np.unique(boun_iter)
    for kk in range(len(boun_unique)):
        find1 = np.where(boun_unique[kk]==boun_iter)[0]
        if len(find1) < pt_conv: # delete the data if only less than 5 data converge to it
            boun_iter = np.delete(boun_iter,find1)
    data_inv_recurrent[str(j)] = data_final[:,boun_iter]
    boun_iter_unstable = boun_iter
    boun_iter_all[str(j)] = np.array(boun_iter_unstable)
    
    # find stable points on the boundary
    data_final1 = F(data_final)
    if kron_on: # use kron when final data not too large
        big_norm = np.linalg.norm(np.kron(data_final1[:2,:],np.ones((1,data_len))) - np.kron(np.ones((1,data_len)),data_final[:2,:]),axis=0)
        minimum = np.min(np.reshape(big_norm,(data_len,data_len)),axis=1)
        boun_map = np.argmin(np.reshape(big_norm,(data_len,data_len)),axis=1).astype('float')
    else: # use for loop when data too large for kron
        def boun_func(kron_iter):
            forward_norm = np.linalg.norm(data_final1[:2,kron_iter]-np.transpose(data_final[:2,:]),axis=1)
            return np.min(forward_norm),np.argmin(forward_norm)
        inputs = tqdm(range(data_len))
        if __name__ == "__main__":
            processed_list = np.array(Parallel(n_jobs=num_cores)(delayed(boun_func)(i) for i in inputs))
        minimum = processed_list[:,0]
        boun_map = processed_list[:,1]
            
    boun_map[minimum>tolerance] = np.nan # conclude that points go to interior if minimum distance with boundary point more than tolerance
    boun_iter = boun_map[boun_map==boun_map].astype('int')
    boun_len = len(np.unique(boun_iter))
    for p in range(traj_iter):
        boun_iter = boun_map[boun_iter]
        boun_iter = boun_iter[boun_iter==boun_iter].astype('int')
        if len(np.unique(boun_iter)) == boun_len:
            break
        boun_len = len(np.unique(boun_iter))
        
    # Record data for plotting
    bounmap_all[str(j)] = boun_map
    boun_unique = np.unique(boun_iter)
    for kk in range(len(boun_unique)):
        find1 = np.where(boun_unique[kk]==boun_iter)[0]
        if len(find1) < pt_conv:
            boun_iter = np.delete(boun_iter,find1)
    data_recurrent[str(j)] = data_final[:,boun_iter]
    
    
    # print steps
    #if j%1==0:
    #    print('step ' +str(j+1))
    if iter1 !=1:
        a_sub += (a_final-a_ini)/(iter1-1)
    print('a = {}'.format(a_sub))
    
# For animation of the boundary tracking    
# fig = plt.figure()
# ax = plt.axes(xlim=(-0.1, 0.1), ylim=(-0.1, 0.1)) #2D
# line, = ax.plot([], [],'.',markersize=1)
# line1, = ax.plot([], [],'.',markersize=1)
# line2, = ax.plot([], [],'.',markersize=10)
# line3, = ax.plot([], [],'.',markersize=5)
# title = ax.text(0.5,0.95, '', bbox={'facecolor':'w', 'alpha':0.5, 'pad':5},transform=ax.transAxes, ha="center")

# # initialise the plot
# def init():
#     line.set_data([],[])
#     line1.set_data([],[])
#     line2.set_data([],[])
#     line3.set_data([],[])
#     return line,

# # animate the plot
# def animate(i):
#     title.set_text(round(a_ini+(a_final-a_ini)*i/(iter-1),3))
#     data = all_data[str(i)]
#     image = all_image[str(i)]
#     pts_inv = data_inv_recurrent[str(i)]
#     pts = data_recurrent[str(i)]
#     line.set_data(data[0,:], data[1,:])
#     line1.set_data(image[0,:], image[1,:])
#     line2.set_data(pts[0,:],pts[1,:])
#     line3.set_data(pts_inv[0,:],pts_inv[1,:])
#     return line, line1, line2, line2, title, 

# # set limit of axes
# ax.set_xlim(-1.7,1.7)
# ax.set_ylim(-0.65,0.65)
# anim = animation.FuncAnimation(fig, animate,
#                                frames=len(all_data), interval=100, blit=True)
# #plt.plot(data_boun[0,:],data_boun[1,:])
# #plt.plot(image[0,:],image[1,:])
# plt.show()

end_time = time.time()
print('total time is {:9.4} seconds'.format(end_time-start_time))


In [ ]:
%matplotlib qt


#plt.plot(data_boun[0,miss_ind],data_boun[1,miss_ind],'.',markersize=20)
# plt.plot(data_boun[0,:],data_boun[1,:])            
plt.plot(image[0,:],image[1,:],'.',markersize=1)
# plt.plot(data_boun[0,np.int32(immune)],data_boun[1,np.int32(immune)],'.',markersize=10)
plt.plot(data_boun[0,:],data_boun[1,:],'.')
plt.plot(data_boun[0,:],data_boun[1,:])        
plt.plot(data_boun[0,dist2],data_boun[1,dist2],'.')  
plt.title('Step {} at a = {}'.format(i,a_sub))
plt.axis('equal')
plt.show()
dist_data = np.linalg.norm(data_boun1[:,1:]-data_boun1[:,:-1],axis=0)
miss_ind = np.where(dist_data>0.004)[0]
print(miss_ind)

In [ ]:
# Show the unstable and stablle fixed or periodic points on the boundary 

%matplotlib qt
i = 0
fig = plt.figure(4)
a_sub = a_ini + (a_final-a_ini)/(max(1,iter1-1))*i
data_final = all_data[str(i)]
image1 = all_image[str(i)]
pts = data_recurrent[str(i)]
pts_inv = data_inv_recurrent[str(i)]
plt.plot(data_final[0,:],data_final[1,:],'.',markersize=.5) # final boundary (MIS approximation)
plt.plot(image1[0,:],image1[1,:],'.',markersize=.5) # image of the bounndary under original mapping
plt.plot(pts_inv[0,:],pts_inv[1,:],'b.',markersize=5) # unstable points on the boundary
plt.plot(pts[0,:],pts[1,:],'g.',markersize=5) # stable points on the boundary
plt.quiver(pts_inv[0,:],pts_inv[1,:],pts_inv[2,:],pts_inv[3,:],scale=30) # arrow on the boundary showing outward pointing normal vector
# plot specific periodic point
# vecvec = np.unique(pts_inv,axis=1)
# vecvec = vecvec[:,[0]]
# plt.plot(vecvec[0,:],vecvec[1,:],'r.',markersize=5)
# Fvecvec = F_n(vecvec,7)
# plt.plot(Fvecvec[0,:],Fvecvec[1,:],'r.',markersize=5)
# Fvecvec = F_n(vecvec,14)
# plt.plot(Fvecvec[0,:],Fvecvec[1,:],'r.',markersize=5)

plt.axis('equal')
plt.title('MIS boundary at a = {}, b = {}, epsilon = {}'.format(np.round(a_sub,7),b_sub,epsilon))
plt.xlabel('x')
plt.ylabel('y')
# plt.xlim(-1,0.8)
# plt.ylim(0.2,0.8)
plt.axis('equal')
# plt.savefig('henon_outward_method_a0608.png',dpi=1000) # save figure 

In [ ]:
# plot the last few iterations of the approximation
vec_record = {}
j = 1
lastfew = 7
for i in range(len(all_data_iter)-lastfew,len(all_data_iter)):
    data = all_data_iter[str(i)]
    image = all_image_iter[str(i)]
    vec_record[j] = data[:2,:]
    if i >= len(all_data_iter)-1:
        plt.scatter(data[0,:],data[1,:],color = 'black',s=.01)
        plt.plot(data[0,:],data[1,:],'.k',markersize=.01)
    else:
        plt.scatter(data[0,:],data[1,:],s=.01)
        plt.plot(data[0,:],data[1,:],'.',markersize=.01)
    plt.title(a_sub)
    plt.axis('equal')
    plt.show()
    j += 1

In [ ]:
# plot the results as all of the iterations (individual = 0) or
# plot the results as last few iterations and map the periodic points and nearby boundary point
# forward and backward using the boundary mapping to show its image and pre image
%matplotlib notebook
individual = 1

if individual:
    i = 0
    if iter1 != 1:
        a_sub = a_ini+(a_final-a_ini)*i/(iter1-1)
    boun_iter_unstable = boun_iter_all[str(i)]
    boun_iter_unstable = np.unique(boun_iter_unstable)
    data = all_data[str(i)]
    image = all_image[str(i)]
    data2 = F_n(data,1)
    pts_inv = data_inv_recurrent[str(i)]
    plt.plot(data[0,:],data[1,:],'.',markersize =1)
    plt.plot(image[0,:],image[1,:],'.',markersize =1)
    plt.plot(pts_inv[0,:],pts_inv[1,:],'.',markersize = 20)
    data4 = F_n(pts_inv,1)
    data3 = F_invn(data,1)
    kk = [k for k in range(len(boun_iter_unstable))] # plot some of the periodic points
    # plot the preimage and image of the found periodic points 
    plt.plot(data2[0,boun_iter_unstable[kk]],data2[1,boun_iter_unstable[kk]],'k.')
    plt.plot(data3[0,boun_iter_unstable[kk]],data3[1,boun_iter_unstable[kk]],'c.')
    if boun_iter_unstable[kk[-1]]+1 == np.shape(data2)[1]: # if the target index is the last
        plt.plot(data2[0,boun_iter_unstable[kk[:-1]]+1],data2[1,boun_iter_unstable[kk[:-1]]+1],'k.')  
        plt.plot(data3[0,boun_iter_unstable[kk[:-1]]+1],data3[1,boun_iter_unstable[kk[:-1]]+1],'c.')
        plt.plot(data2[0,0],data2[1,0],'k.')  
        plt.plot(data3[0,0],data3[1,0],'k.')  
    else:
        plt.plot(data2[0,boun_iter_unstable[kk]+1],data2[1,boun_iter_unstable[kk]+1],'k.')
        plt.plot(data3[0,boun_iter_unstable[kk]+1],data3[1,boun_iter_unstable[kk]+1],'c.')
    plt.plot(data2[0,boun_iter_unstable[kk]-1],data2[1,boun_iter_unstable[kk]-1],'k.')
    plt.plot(data3[0,boun_iter_unstable[kk]-1],data3[1,boun_iter_unstable[kk]-1],'c.')
    # join the inverse point with the forward
#     for ii in range(len(boun_iter_unstable)):
#     for ii in kk: # connect inverse point and forward point representing time-two map
#         plt.plot([data3[0,boun_iter_unstable[ii]],data2[0,boun_iter_unstable[ii]]],[data3[1,boun_iter_unstable[ii]],data2[1,boun_iter_unstable[ii]]])
    for ii in kk: # connect point and forward point representing time-one map
        plt.plot([data[0,boun_iter_unstable[ii]],data2[0,boun_iter_unstable[ii]]],[data[1,boun_iter_unstable[ii]],data2[1,boun_iter_unstable[ii]]])
    plt.title(a_sub)
    if iter1 ==1:
        plt.title('a = {}, b = {}, epsilon = {}'.format(a_sub,b_sub,epsilon))
    else:
        plt.title('a = {}, b = {}, epsilon = {}'.format(round(a_ini+(a_final-a_ini)*i/(iter1-1),7),b_sub,epsilon))

    plt.axis('equal')
else:
    for i in range(len(all_data_iter)):
        data = all_data_iter[str(i)]
        image = all_image_iter[str(i)]
        if i >= len(all_data_iter)-1:
            plt.scatter(data[0,:],data[1,:],color = 'black',s=.001)
            plt.plot(data[0,:],data[1,:],'.k')
        else:
            plt.scatter(data[0,:],data[1,:],s=.001)
            plt.plot(data[0,:],data[1,:],'.')
        plt.title(a_sub)
        plt.axis('equal')
        plt.show()
    epsadd = 0.012
    theta = np.linspace(0,2*np.pi,n_sample)
    plt.scatter(x_ini[0],x_ini[1],color='blue',s=5)
    data_boun = np.array([(epsilon+0.01)*np.cos(theta)+x_ini[0],(epsilon+0.01)*np.sin(theta)+x_ini[1],np.cos(theta),np.sin(theta)])
    plt.scatter(data_boun[0,:],data_boun[1,:],color='cyan',s=.001)
    plt.title('a = {}, b = {}, epsilon = {}'.format(a_sub,b_sub,epsilon))
    plt.xlabel('x')
    plt.ylabel('y')
    plt.axis('equal')
    plt.show()
#     plt.savefig('ooutward_method_a0608_all_iter.png',dpi=1000) # save figure

In [ ]:
# plot an animation of all the parameter regime
import ffmpeg
from IPython import display
from matplotlib import animation
%matplotlib notebook

fig = plt.figure()
ax = plt.axes(xlim=(-0.1, 0.1), ylim=(-0.1, 0.1)) #2D
line, = ax.plot([], [],'.',markersize=1)
line1, = ax.plot([], [],'.',markersize=1)
line2, = ax.plot([], [],'.',markersize=10)
line3, = ax.plot([], [],'.',markersize=5)
title = ax.text(0.5,0.95, '', bbox={'facecolor':'w', 'alpha':0.5, 'pad':5},transform=ax.transAxes, ha="center")

# initialise the plot
def init():
    line.set_data([],[])
    line1.set_data([],[])
    line2.set_data([],[])
    line3.set_data([],[])
    return line,

# animate the plot
def animate(i):
    title.set_text('a = {:.10f}'.format(round(a_ini+(a_final-a_ini)*i/(iter1-1),10)))
    data = all_data[str(i)]
    image = all_image[str(i)]
    pts_inv = data_inv_recurrent[str(i)]
    pts = data_recurrent[str(i)]
    line.set_data(data[0,:], data[1,:])
    line1.set_data(image[0,:], image[1,:])
    line3.set_data(pts_inv[0,:],pts_inv[1,:])
    return line, line1, line2, line3, title, 

# set limit of axes
ax.set_xlim(-0.7,1.6)
ax.set_ylim(-0.65,1.05)
ax.axis('equal')
anim = animation.FuncAnimation(fig, animate,
                               frames=len(all_data), interval=300, blit=True)
plt.show()
Writer = animation.writers['ffmpeg']
writer = Writer(fps=10, metadata=dict(artist='Me'), bitrate=19000)
#writervideo = animation.FFMpegWriter(fps=60)
# anim.save('zoom_henon_outward_method_06076-060775.mp4',writer=writer,dpi=300) # save figure